## Validation 和 Cross Validation

In [10]:
import numpy as np
from sklearn import datasets

In [14]:
digits = datasets.load_digits()
X = digits.data
y = digits.target

### 测试train_test_split

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=666)

In [20]:
from sklearn.neighbors import KNeighborsClassifier

# 对k(邻居)，p(距离的计算方式)进行调参
best_k, best_p, best_score = 0, 0, 0
for k in range(2, 11):
    for p in range(1, 6):
        knn_clf = KNeighborsClassifier(weights="distance", n_neighbors=k, p=p)
        knn_clf.fit(X_train, y_train)
        score = knn_clf.score(X_test, y_test)
        if score > best_score:
            best_k, best_p, best_score = k, p, score
            
print("Best K =", best_k)
print("Best P =", best_p)
print("Best Score =", best_score)

Best K = 3
Best P = 4
Best Score = 0.9860917941585535


### 使用交叉验证

In [11]:
from sklearn.model_selection import cross_val_score

knn_clf = KNeighborsClassifier()
# 默认就是对X_train分成5份进行验证
cross_val_score(knn_clf, X_train, y_train)

array([0.99537037, 0.98148148, 0.97685185, 0.97674419, 0.97209302])

In [6]:
best_k, best_p, best_score = 0, 0, 0
for k in range(2, 11):
    for p in range(1, 6):
        knn_clf = KNeighborsClassifier(weights="distance", n_neighbors=k, p=p)
        # 直接使用CVscore计算即可
        scores = cross_val_score(knn_clf, X_train, y_train)
        score = np.mean(scores)
        if score > best_score:
            best_k, best_p, best_score = k, p, score
            
# 使用CV得到的参数更加可信。因为手动的train-test-split的有可能只是对test存在过拟合现象
# 得到的best_score并不是最终评价该模型的分数
# 拿到这组K和P之后得到的knnclassifier求出score即可
print("Best K =", best_k)
print("Best P =", best_p)
print("Best Score =", best_score)

Best K = 2
Best P = 2
Best Score = 0.9851507321274763


In [7]:
best_knn_clf = KNeighborsClassifier(weights="distance", n_neighbors=2, p=2)
best_knn_clf.fit(X_train, y_train)
# test是在网格搜索中完全没有使用的，所以得到的结果更加可信
best_knn_clf.score(X_test, y_test)

0.980528511821975

### 回顾网格搜索

In [8]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {
        'weights': ['distance'],
        'n_neighbors': [i for i in range(2, 11)], 
        'p': [i for i in range(1, 6)]
    }
]

# 网格搜索的GridSearchCV就是使用CV进行搜索的
grid_search = GridSearchCV(knn_clf, param_grid, verbose=1)
grid_search.fit(X_train, y_train)

# 225 = 5*45

Fitting 5 folds for each of 45 candidates, totalling 225 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 225 out of 225 | elapsed:   34.6s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=10, p=5,
                                            weights='distance'),
             iid='deprecated', n_jobs=None,
             param_grid=[{'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10],
                          'p': [1, 2, 3, 4, 5], 'weights': ['distance']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=1)

In [12]:
grid_search.best_score_

0.9851507321274763

In [15]:
grid_search.best_params_

{'n_neighbors': 2, 'p': 2, 'weights': 'distance'}

In [18]:
best_knn_clf = grid_search.best_estimator_
best_knn_clf.score(X_test, y_test)

0.980528511821975

### cv参数

In [23]:
cross_val_score(knn_clf, X_train, y_train, cv=10)

array([0.99074074, 1.        , 0.98148148, 0.9537037 , 0.99074074,
       0.96296296, 0.94444444, 0.98148148, 0.98130841, 0.98130841])

In [25]:
grid_search = GridSearchCV(knn_clf, param_grid, verbose=1, cv=10)
grid_search.fit(X_train, y_train)

neighbors=8, p=1, weights=distance, total=   0.0s
[CV] n_neighbors=8, p=2, weights=distance ............................
[CV] ............. n_neighbors=8, p=2, weights=distance, total=   0.0s
[CV] n_neighbors=8, p=2, weights=distance ............................
[CV] ............. n_neighbors=8, p=2, weights=distance, total=   0.0s
[CV] n_neighbors=8, p=2, weights=distance ............................
[CV] ............. n_neighbors=8, p=2, weights=distance, total=   0.0s
[CV] n_neighbors=8, p=2, weights=distance ............................
[CV] ............. n_neighbors=8, p=2, weights=distance, total=   0.0s
[CV] n_neighbors=8, p=2, weights=distance ............................
[CV] ............. n_neighbors=8, p=2, weights=distance, total=   0.0s
[CV] n_neighbors=8, p=2, weights=distance ............................
[CV] ............. n_neighbors=8, p=2, weights=distance, total=   0.0s
[CV] n_neighbors=8, p=2, weights=distance ............................
[CV] ............. n_neighb

GridSearchCV(cv=10, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=10, p=5,
                                            weights='distance'),
             iid='deprecated', n_jobs=None,
             param_grid=[{'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10],
                          'p': [1, 2, 3, 4, 5], 'weights': ['distance']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=2)